In [132]:
import pandas as pd
import numpy as np
import sklearn.preprocessing, sklearn.ensemble, sklearn.feature_selection, sklearn.model_selection 
import sklearn.metrics, sklearn.linear_model, sklearn.pipeline

In [134]:
df = pd.read_csv("dataset.csv")
df = df.iloc[:, :-1]
df["filename"] = df["filename"].str[:-4]

target = pd.read_csv("targets.tsv", sep="\t")

df = df.merge(target)

Selecting features and preprocessing

In [67]:
estimator = AdaBoostClassifier(learning_rate=1, n_estimators=100)
selector = RFECV(estimator, n_jobs=-1)
X = df.iloc[:, 1:-1]
Y = df.iloc[:, -1]
X_new = selector.fit_transform(X, Y)

In [123]:
sc = StandardScaler()
X_sd_new = sc.fit_transform(X_new)

In [69]:
selector.ranking_

array([ 1,  8,  1,  1,  6,  1,  1,  5,  1,  1,  1,  1,  1,  4,  1,  1,  2,
       10,  1,  1,  1,  1,  1,  3,  1,  1,  1,  1, 11,  9,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  7,  1])

In [135]:
selector.support_
X_sd_new.shape

(13936, 35)

In [71]:
x_train, x_test, y_train, y_test = train_test_split(X_sd_new, Y, test_size=0.05, random_state=0)

In [72]:
rd_forest = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=15, criterion="entropy")
logit = LogisticRegression(C=100)
clf = AdaBoostClassifier(estimator=logit, learning_rate=0.5, n_estimators=100)
SVM = SGDClassifier()

In [73]:
rd_forest.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=15, n_jobs=-1)

In [74]:
logit.fit(x_train, y_train)

LogisticRegression(C=100)

In [75]:
clf.fit(x_train, y_train)

AdaBoostClassifier(estimator=LogisticRegression(C=100), learning_rate=0.5,
                   n_estimators=100)

In [76]:
SVM.fit(x_train, y_train)

SGDClassifier()

In [77]:
y_pred_clf = clf.predict(x_test)
y_pred_logit = logit.predict(x_test)
y_pred_rd_forest = rd_forest.predict(x_test)
y_pred_SVM = SVM.predict(x_test)

In [127]:
print(classification_report(y_test, y_pred_clf))

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       369
           1       0.95      0.90      0.92       328

    accuracy                           0.93       697
   macro avg       0.93      0.93      0.93       697
weighted avg       0.93      0.93      0.93       697



In [128]:
print(classification_report(y_test, y_pred_logit))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95       369
           1       0.95      0.92      0.94       328

    accuracy                           0.94       697
   macro avg       0.94      0.94      0.94       697
weighted avg       0.94      0.94      0.94       697



In [129]:
print(classification_report(y_test, y_pred_rd_forest))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       369
           1       0.96      0.93      0.94       328

    accuracy                           0.95       697
   macro avg       0.95      0.95      0.95       697
weighted avg       0.95      0.95      0.95       697



In [130]:
print(classification_report(y_test, y_pred_SVM))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91       369
           1       0.93      0.87      0.90       328

    accuracy                           0.91       697
   macro avg       0.91      0.90      0.91       697
weighted avg       0.91      0.91      0.91       697



In [131]:
df_result = pd.read_csv("test.csv")
X_result = df_result.iloc[:, 1:-1]

In [88]:
X_result

,mfcc_mean1,mfcc_mean2,mfcc_mean3,mfcc_mean4,mfcc_mean5,mfcc_mean6,mfcc_mean7,mfcc_mean8,mfcc_mean9,mfcc_mean10,...,mfcc_std16,mfcc_std17,mfcc_std18,mfcc_std19,mfcc_std20,cent_mean,cent_std,cent_skew,rolloff_mean,rolloff_std
0,-495.62723,81.183710,1.594539,29.115295,-4.104828,7.492647,-1.469902,7.632004,-3.318697,3.421593,...,10.342658,5.448217,6.600827,5.392629,5.709760,1651.372555,1064.234980,1.429588,3380.210449,1643.595855
1,-601.48920,77.305840,26.046585,27.844090,1.885911,5.209487,-3.427303,5.228904,0.742163,5.641624,...,7.781601,5.004192,6.337066,5.048322,5.901637,1464.731393,892.500959,1.907080,3261.781820,1604.861048
2,-510.97247,94.028206,-0.028242,27.325160,-7.126594,6.687743,-5.592118,6.012521,3.481522,3.813088,...,6.337800,5.082880,5.494977,7.392568,5.420835,1608.648572,1043.918670,0.621692,3317.936657,2053.706874
3,-577.55164,64.389244,17.996605,21.222319,-0.751681,8.161698,2.665672,8.443647,6.687653,5.114984,...,7.964971,4.464272,6.431951,3.316650,3.276715,1507.763557,894.455457,1.977395,3228.136190,1380.720097
4,-504.74970,110.945465,-6.109226,26.858955,-8.610896,9.381459,-4.851646,7.885846,-1.336644,0.231714,...,7.798015,7.101612,5.459076,4.190422,3.937598,1498.658161,1221.100004,1.419302,2925.496952,2037.058329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3408,-523.24554,69.272390,11.020937,27.752268,3.869794,10.757477,-1.150927,9.536394,0.071508,2.150174,...,5.801632,3.748371,6.873589,4.775026,5.597802,1723.557587,979.400600,1.293535,3641.852824,1599.072758
3409,-434.61786,82.055170,-7.900979,22.883240,0.809252,5.232460,0.446307,4.846217,-8.800628,-0.780675,...,7.622484,8.251905,5.372429,5.639769,4.639079,1485.664638,940.663231,1.257874,2952.471313,1585.285630
3410,-503.56990,25.719360,6.686260,16.090012,3.362235,9.102875,-3.528343,1.984147,-0.272554,2.024660,...,6.683687,4.462136,5.568662,3.520132,3.743933,1685.867292,907.012982,1.677187,3786.205037,1490.614635
3411,-584.67600,64.407670,25.802309,27.096209,7.304740,10.648228,1.088120,4.132356,0.491814,2.911443,...,5.654293,3.347367,4.084435,3.288080,3.617672,1441.681768,812.969990,1.937356,3311.726888,1407.112886


In [89]:
X_result_new = X_result.loc[:, selector.support_]

In [91]:
X_result_sd_new = sc.transform(X_result_new)

M:\Anaconda\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


(3413, 35)

In [92]:
label_result = logit.predict(X_result_sd_new)

In [121]:
result = df_result
result["target"] = label_result
result = result.loc[:, ["filename", "target"]]
result["filename"] = result["filename"].str[:-4]
result.to_csv("result.tsv", sep="\t", index=False)
result

,filename,target
0,00100026dbdffcd01cde6ee9b9a9d273,1
1,0014278276a6cc05fe8c522af0a677df,1
2,0026e20266ceba9cdda2c116e89d2f3b,1
3,008b37fc832d3dfad9105961c5801c02,1
4,00b1d7d0d6fdb25050041f6e2ae2871e,0
...,...,...
3408,ffbea6a332d113f0bd16cf0b4ed07dc6,0
3409,ffee25334e0247dd07d80d0eb3ce7886,0
3410,fff86627949b37d236237ff895ff4268,0
3411,fff8aa9bac38e85734f64aad2b2f3814,0


In [ ]:
header = ['filename']
header.append("target")

with open('result.tsv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t', lineterminator="\n")
    writer.writerow(header)
    for 

In [40]:
target

,filename,target
0,9a701a4536a05b6610a590a9fe702ed8,1
1,cad0b8547008d1524c1a0e5fd51f9908,1
2,4bbe607e7dc95460e2cc1a6ee5f4dfa6,0
3,30fb32cba90b34af26f3f14f5d636805,0
4,fa33445afe71a6dc18e4881c053da5be,0
...,...,...
13931,d26e5ebee3589530a4ec735ba1baab83,0
13932,b9ae9428d4d7d279068aea06365d5736,1
13933,3da22010bcdd5b47d7f2d3ceea190195,0
13934,f8ef0441ebf79b817df790bb72d667c4,0
